In [1]:
# Install the pycoco library
!pip install pycocotools

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272449 sha256=3aba2675c63e9bce94d6b8ba0883ba7cb25db552742b2a0db5592f6d43851916
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [2]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import random
import cv2
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
import imgaug.augmenters as iaa

In [3]:
# Define the image and annotation paths
train_imgs_path = '../input/food-recognition-challenge/train-v0.4/train/images'
train_anns_path = '../input/food-recognition-challenge/train-v0.4/train/annotations.json'

val_imgs_path = '../input/food-recognition-challenge/val-v0.4/val/images'
val_anns_path = '../input/food-recognition-challenge/val-v0.4/val/annotations.json'

test_imgs_path = '../input/food-recognition-challenge/test_images-v0.4/val/images'
test_anns_path = '../input/food-recognition-challenge/test_images-v0.4/val/annotations.json'

# Reference to Image Segmentation

https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html

In [4]:
# Function to load JSON into a COCO api
def getCOCO(anns_path):
    # Initialize the COCO api for instance annotations
    coco = COCO(anns_path)
    
    # Load the categories in a variable
    catIDs = coco.getCatIds()
    cats = coco.loadCats(catIDs)
    
    # Print number of categories
    nms = [cat['name'] for cat in cats]
    print('\nNumber of COCO categories: {}'.format(len(nms)))
    
    # Create a dataframe of the count of each category
    df = pd.DataFrame(columns=['Category', 'ID', 'Count'])
    
    # Add each category and its count row by row
    for i, catID in enumerate(catIDs):
        imgIds = coco.getImgIds(catIds=catID)
        df.loc[i] = [nms[i]] + [catID] + [len(imgIds)]     
        
    return coco, df

In [5]:
# Load the annotations of the image dataset
coco, classes = getCOCO(train_anns_path)

# Preview a sample of the classes dataframe
classes.head()

loading annotations into memory...
Done (t=4.31s)
creating index...
index created!

Number of COCO categories: 273


,Category,ID,Count
0,water,2578,1835
1,pear,1157,151
2,egg,2022,626
3,grapes,1198,94
4,butter,2053,1008


# Reference for Getting Masks

https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-2-of-2-c0d1f593096a

https://github.com/virafpatrawala/COCO-Semantic-Segmentation/blob/master/COCOdataset_SemanticSegmentation_Demo.ipynb

https://github.com/qubvel/segmentation_models/issues/137

https://www.reddit.com/r/computervision/comments/ihh8n0/onehotencoding_with_multichannel_images/

In [6]:
def getGenerator(img_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)
    
    while (True):
        img_batch = np.zeros((batch_size, 224, 224, 3)).astype('int')
        mask_batch = np.zeros((batch_size, 224, 224, 274)).astype('int')

        for i in range(c, c + batch_size):
            train_img = cv2.imread(img_folder + '/' + n[i])
            train_img =  cv2.resize(train_img, (224, 224))
            train_img = cv2.cvtColor(train_img, cv2.COLOR_BGR2RGB)
            img_batch[i-c] = train_img

            def getMask(image_id):
                # Create a zero array with the given size and number of classes
                mask = np.zeros((224, 224, 274))
            
                annIds = coco.getAnnIds(int(image_id))
                anns = coco.loadAnns(annIds)
                
                for j, ann in enumerate(anns):
                    # Get the binary mask for the annotation
                    binary = cv2.resize(coco.annToMask(ann), (224, 224))

                    # Get the channel index for the annotation
                    channel = classes[classes.ID == ann['category_id']].index[0] + 1

                    # Update the channel of the annotation
                    mask[:, :, channel] = binary
                    
                    # Update the background channel of the annotation
                    if j == 0:
                        mask[:, :, 0] = np.logical_or(mask[:, :, 0], np.logical_not(binary))
                    else:
                        mask[:, :, 0] = np.logical_and(mask[:, :, 0], np.logical_not(binary))

                return mask
            
            train_mask = getMask(n[i][1:-4])

            mask_batch[i-c] = train_mask

        c += batch_size
        if(c + batch_size >= len(os.listdir(img_folder))):
            c = 0
            random.shuffle(n)

        yield img_batch, mask_batch

In [7]:
# Get the images and masks from the train path
train_gen = getGenerator(train_imgs_path, batch_size=10)

# Get the images and masks from the validation path
val_gen = getGenerator(val_imgs_path, batch_size=10)

# Reference for Model Building

https://github.com/qubvel/segmentation_models#quick-start

https://github.com/qubvel/segmentation_models/issues/374

https://github.com/qubvel/segmentation_models/blob/master/examples/multiclass%20segmentation%20(camvid).ipynb

https://github.com/qubvel/segmentation_models/issues/254

In [8]:
# Install and import the segmentation models library
!pip install segmentation_models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

     |████████████████████████████████| 50 kB 650 kB/s 
env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [9]:
# Define the model to be trained
model = sm.Unet('resnet34', encoder_weights='imagenet', classes=274, activation='softmax')
model.compile('Adam', loss='categorical_crossentropy', metrics=[sm.metrics.iou_score, sm.metrics.precision, sm.metrics.recall])

85524480/85521592 [==============================] - 4s 0us/step


In [10]:
# Train the defined model on the dataset
history = model.fit(train_gen, epochs=5, steps_per_epoch=10)

Epoch 1/5
10/10 [==============================] - 126s 11s/step - loss: 5.2299 - iou_score: 1.7665e-04 - precision: 0.0037 - recall: 0.9433
Epoch 2/5
10/10 [==============================] - 106s 11s/step - loss: 4.2259 - iou_score: 3.5051e-04 - precision: 0.0044 - recall: 0.9457
Epoch 3/5
10/10 [==============================] - 106s 11s/step - loss: 3.0664 - iou_score: 0.0011 - precision: 0.0045 - recall: 0.9428
Epoch 4/5
10/10 [==============================] - 105s 11s/step - loss: 2.6609 - iou_score: 0.0020 - precision: 0.0051 - recall: 0.9465
Epoch 5/5
10/10 [==============================] - 110s 11s/step - loss: 1.8844 - iou_score: 0.0022 - precision: 0.0045 - recall: 0.9543
